<a href="https://colab.research.google.com/github/ekapolc/exxon_training/blob/master/explainability/ExxonTraining2_Classification_Shap_Calibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install these packages before starting this lab

In [1]:
!pip install xlrd
!pip install pydot
!pip install pyparsing
!apt-get install graphviz
!pip install graphviz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
graphviz is already the newest version (2.40.1-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


## Import libraries

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from xgboost import XGBClassifier, plot_tree, plot_importance
import seaborn as sns; sns.set()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
np.version.version

'1.16.4'

# XGboost

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
# Load our pre-processed data
PATH = '/content/'
df = pd.read_csv(PATH+'data.csv')

In [6]:
print(df.shape)
df.head()

(4591, 19)


,SerialNumber,Leave,ActionYear,WorkDurationYear,CountLoan,Avg_MonthPerLoan,HireType,HireSourceGroup,WorkDurationYear.1,Avg_TotalAbsensePerYear,Avg_NumDaysPerAbsense,TotalEduAllowance,NumYear_SinceLastEduAllowance,TotalEduAttend,EduBranch_CHEM,EduBranch_Finance,EduBranch_Languages,Max_EduInstituteGroup,NumYear_SinceLastEdu
0,4,1.0,2000,39.0,0.0,0.0,Unknown,Unknown,39.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Unknown,41.0
1,5,1.0,2000,39.0,0.0,0.0,Unknown,Unknown,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UNIV,40.0
2,6,1.0,2000,38.0,0.0,0.0,Unknown,Unknown,38.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Unknown,47.0
3,7,1.0,2000,38.0,0.0,0.0,Unknown,Unknown,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SCHL,39.0
4,10,1.0,2000,38.0,0.0,0.0,Unknown,Unknown,38.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Unknown,38.0


In [0]:
# Print columns in dataframe
print(df.columns.values)

['SerialNumber' 'Leave' 'ActionYear' 'WorkDurationYear' 'CountLoan'
 'Avg_MonthPerLoan' 'HireType' 'HireSourceGroup' 'WorkDurationYear.1'
 'Avg_TotalAbsensePerYear' 'Avg_NumDaysPerAbsense' 'TotalEduAllowance'
 'NumYear_SinceLastEduAllowance' 'TotalEduAttend' 'EduBranch_CHEM'
 'EduBranch_Finance' 'EduBranch_Languages' 'Max_EduInstituteGroup'
 'NumYear_SinceLastEdu']


### Todo#1 - Create One-hot features
In the previous lab, we have learned how to deal with categorical features. This lab also contains categorical features, and we will be dealing with them.

#### Question : Explore data and find out which features are categorical.

#### Answer : 

#### Instructions : Write code to transform the categorical features to one-hot.

In [0]:
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

# Create one-hot feature columns. We will drop the original columns afterwards.


# Concatenate the new columns to the dataframe


<details>
    <summary>SOLUTION HERE!</summary>
      <pre>
        <code>
dfEduInstituteGroup = pd.get_dummies(df['Max_EduInstituteGroup'], prefix='Max_EduInstituteGroup')
dfHireTypeGroup = pd.get_dummies(df['HireType'], prefix='HireType')
dfHireSourceGroup = pd.get_dummies(df['HireSourceGroup'], prefix='HireSourceGroup')

df = pd.concat([df, dfEduInstituteGroup, dfHireTypeGroup, dfHireSourceGroup], axis=1)
        </code>
      </pre>
</details>

In [0]:
df.head()

### Todo#2 - Split Train and Test

Usually, time series data will be splitted to train and test set according to time period, to avoid the look-ahead bias -- taking data from the "future" which we are not supposed to know yet. To simulate a production environment, the test set should be the most recent part of data. 

We will try to use xgboost model to predict whether an employee is still working for the company after 2017. So, we divide our train and test set using ActionYear 2017.

#### Instructions : Split the data into `df_train` for training and `df_test` for testing. Use data *before* 2017 as the train set, and the rest (from 2017 and after) as the test set.

In [0]:
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

### Split train and test data using "ActionYear" feature


<details>
<summary>SOLUTION HERE!</summary>
<pre>
<code>
df_train = df[ df['ActionYear'] < 2017]
df_test = df[df['ActionYear'] >= 2017]

</code>
</pre>
</details>

In [0]:

df_train.shape, df_test.shape

###Todo#3 - Drop irrelevant columns and create labels

#### Question : Besides the original categorical columns, which columns do you think are not necessary for your model?

#### Answer :

#### Instructions : Drop all unnecessary columns, including the categorical columns from Todo#1 and the columns you answered in this question. Also, put the labels (did the employee leave the company or not) into variables `df_train_label` and  `df_test_label`.

In [0]:
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

### Drop irrelevant columns and build label in df_train 


### Drop irrelevant columns and build label in df_test 


<details>
<summary>SOLUTION HERE!</summary>
<pre>
<code>
df_train_variable = df_train.drop(['SerialNumber','ActionYear','Leave','Max_EduInstituteGroup','HireType','HireSourceGroup'],axis=1)
df_train_label = df_train['Leave']

df_test_variable = df_test.drop(['SerialNumber','ActionYear','Leave','Max_EduInstituteGroup','HireType','HireSourceGroup'],axis=1)
df_test_label = df_test['Leave']

</code>
</pre>
</details>

In [0]:
### Just change name variable
X_train, X_test, y_train, y_test = df_train_variable, df_test_variable, df_train_label, df_test_label

In [0]:
print(y_train.shape)
print(y_test.shape)

### Todo#4 - Build XGBoost model

Here, we will create our xgboost classifier that will classify 2 classes -- whether each employee will leave the company by the end of 2017 or not -- using some pre-defined hyperparameters. Note that these hyperparameters should be tuned for each different task and dataset.

Full details of each hyperparameter can be found in [XGBoost Docs](https://xgboost.readthedocs.io/en/latest/parameter.html#parameters-for-tree-booster)

#### Instructions : Define your model and set its parameters according to the following:
- n_jobs =16
- n_estimators=400
- max_depth=4
- objective="binary:logistic"
- learning_rate=0.07
- subsample=0.9
- min_child_weight=6
- colsample_bytree=.9
- scale_pos_weight=0.8
- gamma=8
- reg_alpha=6
- reg_lambda=1.3

In [0]:
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

# Define our model


<details>
<summary>SOLUTION HERE!</summary>
<pre>
<code>
model = XGBClassifier(    
    n_jobs=-1,
    n_estimators=400,
    max_depth=4,
    objective="binary:logistic",
    learning_rate=0.07, 
    subsample=0.9,
    min_child_weight=6,
    colsample_bytree=.9,
    scale_pos_weight=0.8,
    gamma=8,
    reg_alpha=6,
    reg_lambda=1.3)

</code>
</pre>
</details>

In [0]:
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

# Train the model

<details>
<summary>SOLUTION HERE!</summary>
<pre>
<code>
model.fit(X_train, y_train)
print(model)
</code>
</pre>
</details>

#### Predict test labels for model evaluation
Just like the previous lab, by calling predict_proba(input_data), the model will output the probabilities of each output class.

In [0]:
model.predict_proba(X_test)

In [0]:
# Predict test label for model evaluation
# Since we only have 2 classes, 0 and 1, we will use predict_proba(X_test)[:,1] as probability of employees levaing the company
y_pred = model.predict_proba(X_test)[:,1]

### set temp variable to use in later cells.
y_test_model1 = y_test
y_predictions_model1 = y_pred

# Round the predicted probabilities to get predicted labels
predictions = [np.round(value) for value in y_pred]

In [0]:
print(classification_report(y_test, predictions))

In [0]:
print(confusion_matrix(y_test, predictions))

----------

### Todo#5 - Target Encoding
When dealing with columns of categorical features (e.g. user group is either 1, 2, or 3) we usually create new one-hot feature columns for all possible categories which will have value '1' in the column that each of them belongs to, and '0' otherwise. However, by creating new one-hot feature columns for every category, we will end up with a lot of columns, which might be difficult for the model to learn.

In this section, we will encode those categorical features into a new columns that will be easier to be learned by the model.

In [0]:
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))


def target_encode(trn_series=None,
                  tst_series=None,
                  target=None,
                  min_samples_leaf=1,
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior
    """
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    
    # Apply average function to all target data
    prior = target.mean()
    
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index

    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

### Train new model with target encoding

In [0]:
# Load our pre-processed data
df = pd.read_csv(PATH+'data.csv')

In [0]:
df.dtypes

#### Instructions : Write code to transform the categorical features to one-hot.

In [0]:
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

# Create one-hot feature columns for Max_EduInstituteGroup, HireType, HireSourceGroup


# Concat the new columns to the dataframe


<details>
  <summary>SOLUTION HERE!</summary>
  <pre>
    <code>
dfEduInstituteGroup = pd.get_dummies(df['Max_EduInstituteGroup'], prefix='Max_EduInstituteGroup')
dfHireTypeGroup = pd.get_dummies(df['HireType'], prefix='HireType')
dfHireSourceGroup = pd.get_dummies(df['HireSourceGroup'], prefix='HireSourceGroup')

df = pd.concat([df, dfEduInstituteGroup,dfHireTypeGroup,dfHireSourceGroup], axis=1)
      </code>
  </pre>
</details>

In [0]:
# Split train/test 
df_train = df[ df['ActionYear'] < 2017]
df_train.shape

df_test = df[ df['ActionYear'] >= 2017]
df_test.shape

df_train_variable = df_train.drop(['SerialNumber','ActionYear','Leave'],axis=1)
df_train_label = df_train['Leave']

df_test_variable = df_test.drop(['SerialNumber','ActionYear','Leave'],axis=1)
df_test_label = df_test['Leave']

X_train, X_test, y_train, y_test = df_train_variable, df_test_variable, df_train_label, df_test_label

In [0]:
# Target encode these categorical features in our data
col_names_cat = ['HireType', 'HireSourceGroup', 'Max_EduInstituteGroup']

In [0]:
# Enocode categorical features
for f in col_names_cat:
    X_train[f + "_avg"], X_test[f + "_avg"] = target_encode(
                                              trn_series=X_train[f],
                                              tst_series=X_test[f],
                                              target=y_train,
                                              min_samples_leaf=200,
                                              smoothing=10,
                                              noise_level=0
                                              )

In [0]:
X_train.columns.values

In [0]:
# Sample encoded values
X_train[:10][['HireType', 'HireType_avg', 'HireSourceGroup', 'HireSourceGroup_avg', 'Max_EduInstituteGroup', 'Max_EduInstituteGroup_avg']]

#### Question : Looking at the cell above, what do you think about encoding these columns? Does it make sense to do? Give your explanation.

#### Answer : 

In [0]:
# Drop the original features
X_train = X_train.drop(['Max_EduInstituteGroup','HireType','HireSourceGroup'], axis=1)
X_test = X_test.drop(['Max_EduInstituteGroup','HireType','HireSourceGroup'], axis=1)

#### Instructions : Define the model with the same parameters as Todo#4.

In [0]:
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

# Define model same as previous model


<details>
  <summary>SOLUTION HERE!</summary>
  <pre>
    <code>
model = XGBClassifier(    
    n_jobs=-1,
    n_estimators=400,
    max_depth=4,
    objective="binary:logistic",
    learning_rate=0.07, 
    subsample=0.9,
    min_child_weight=6,
    colsample_bytree=.9,
    scale_pos_weight=0.8,
    gamma=8,
    reg_alpha=6,
    reg_lambda=1.3)
      </code>
  </pre>
</details>

In [0]:
# Train
model.fit(X_train, y_train)
print(model)

In [0]:
# make predictions for test data
y_pred_all = model.predict_proba(X_test)
y_pred = y_pred_all[:,1]

# set tmp for using in roc 
y_test_model2 = y_test
y_predictions_model2 = y_pred

predictions = [np.round(value) for value in y_pred]

In [0]:
print(classification_report(y_test, predictions))

In [0]:
### Model 1 (Not-Encode)
fpr1, tpr1, thresholds = roc_curve(  y_test_model1, y_predictions_model1 )
roc_auc1 = auc(fpr1, tpr1)

### Model 2 (Encode)
fpr2, tpr2, thresholds = roc_curve(  y_test_model2, y_predictions_model2 )
roc_auc2 = auc(fpr2, tpr2)

plt.figure(figsize=(12,8))
plt.plot(fpr1, tpr1, color='darkorange', lw=1, label='ROC curve (area = %0.2f)' % roc_auc1) ## Orange Line
plt.plot(fpr2, tpr2, lw=1, label='ROC curve (area = %0.2f)' % roc_auc2) ## Blue Line
plt.plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")

plt.show()

### Todo#6 - Visualize our graph model
XGBoost is a gradient boosting model. With graphviz library, you can visually find out how your model works.

In [0]:
# View tree
# Because graphviz seams to have some resolution problem with pyplot and jupyter, we have to use to trick below to get a readable graph
# fig = plt.gcf()
# fig.set_size_inches(10, 10)
fig, ax = plt.subplots(figsize=(10, 10), dpi=200)

# Plot the n-th tree of the model. Note that the model consists of many tress (as defined when creating the model)
plot_tree(model, num_trees=0,  ax=ax)
# fig.savefig('tree.png')

In [0]:
# Plot how many times each feature is used as node in the tree
fig, ax = plt.subplots(figsize=(12, 9))
plot_importance(model, ax=ax)

#### Question : What do you think about the insight and information from the above cells? Does it make sense, and how?
#### Answer : 

### Todo#7 - Deeply explore your  model with Shap

<a href ="https://github.com/slundberg/shap">SHAP</a> (SHapley Additive exPlanations) is a unified approach to explaining the output of any machine learning model. SHAP connects game theory with local explanations, uniting several previous methods and representing the only possible consistent and locally accurate additive feature attribution method based on expectations.

<img src="https://raw.githubusercontent.com/slundberg/shap/master/docs/artwork/shap_diagram.png">

In [0]:
!pip install shap
import shap

In [0]:
explainer = shap.TreeExplainer(model)

In [0]:
shap_values = explainer.shap_values(X_test)
print('Expected Value:', explainer.expected_value)
pd.DataFrame(shap_values).head()

In [0]:
### Google Colab needs shap.initjs() in every cell where there is a visualization.
shap.initjs()
shap.force_plot(explainer.expected_value, 
                shap_values[0,:], X_test.iloc[0,:])

In [0]:
shap.initjs()
shap.force_plot(explainer.expected_value, 
                shap_values, X_test)

In [0]:
shap.summary_plot(shap_values,
                  X_test, plot_type="bar")

In [0]:
shap.summary_plot(shap_values, X_test)

In [0]:
shap.dependence_plot(ind='WorkDurationYear', interaction_index='WorkDurationYear',
                     shap_values=shap_values, 
                     features=X_test,  
                     display_features=X_test)

#### Question : As you explore your model using Shap, is there anything that makes you interested? Try to change your model parameter and see the changed shap plot.
#### Answer : 

### Todo#8 - Calibration with Ensemble
Predicted probabilities that match the expected distribution of probabilities for each class are said to be *calibrated*. However, not all machine learning models are capable of predicting calibrated probabilities. There are methods to both diagnose how calibrated the predicted probabilities are, and how to better calibrate the predicted probabilities with the observed distribution of each class. Often, this can lead to better quality predictions, depending on how the skill of the model is evaluated.

Ensemble techniques can help us calibrate our model.

For more examples on calibration see <a href="https://machinelearningmastery.com/calibrated-classification-model-in-scikit-learn/">here for more details</a>



In [0]:
# Define bins function
def buildBins(y_pred):
  count_bin = [0]*10
  total_bin = [0]*10
  for i in range(len(y_pred)):
    total_bin[(int(y_pred[i][1]*100//10))]+=1
    if(y_test.values[i]==1 and y_pred[i][1]>=0.5):
      count_bin[(int(y_pred[i][1]*100//10))]+=1
  return np.array(count_bin), np.array(total_bin)

def plotCalibrate(count_bin, total_bin):
  
  total_bin[total_bin==0]=1
  ratio_bin = count_bin / total_bin
  fig, ax = plt.subplots(figsize=(15,9), )
  lm = sns.barplot(np.arange(10),ratio_bin,palette="Blues_d")
  sns.lineplot(np.arange(10), np.arange(10)/10)
  axes = lm.axes
  axes.set_ylim(0,1)

In [0]:
# cal bin for simple model
count_bin, total_bin = buildBins(y_pred_all.copy())

In [0]:
# plot number of data in each bins
plt.figure(figsize=(15,9))
sns.barplot(np.arange(10),total_bin, facecolor=(0.9, 0.4, 0.4, 0.5),saturation=0.9)
sns.barplot(np.arange(10),count_bin,facecolor=(0.1, 0.6, 0.7, 0.9))

In [0]:
# plot calibrate on simple model
plotCalibrate(count_bin, total_bin)

In [0]:
### 10 folds and 10 model (ensemble)
kf = KFold(n_splits=10)
model_set = {}
y_pred_ensemble_all = []
tmp = 0
for train_index, test_index in kf.split(X_train):
  model = XGBClassifier(    
          n_jobs=16,
          n_estimators=400,
          max_depth=4,
          objective="binary:logistic",
          learning_rate=0.07, 
          subsample=0.9,
          min_child_weight=6,
          colsample_bytree=.9,
          scale_pos_weight=0.8,
          gamma=8,
          reg_alpha=6,
          reg_lambda=1.3)
  model_set[tmp] = model
  X_train_tmp, _ = X_train.iloc[train_index], X_train.iloc[test_index]
  y_train_tmp, _ = y_train.iloc[train_index], y_train.iloc[test_index]
  model.fit(X_train_tmp, y_train_tmp)
  y_pred_ensemble_all.append(model.predict_proba(X_test))
  tmp+=1

In [0]:
y_pred_ensemble = np.array(y_pred_ensemble_all).mean(axis=0)

In [0]:
count_bin_ensemble, total_bin_ensemble = buildBins(y_pred_ensemble.copy())

In [0]:
plt.figure(figsize=(15,9))
sns.barplot(np.arange(10),total_bin_ensemble, facecolor=(0.9, 0.4, 0.4, 0.5),saturation=0.9)
sns.barplot(np.arange(10),count_bin_ensemble,facecolor=(0.1, 0.6, 0.7, 0.9))

In [0]:
plotCalibrate(count_bin_ensemble,total_bin_ensemble)

In [0]:
def calGap(total, count):
  total[total==0]=1
  ratio = count / total
  return abs(ratio - np.array([0.5,0.15,0.25,0.35,0.45,0.55,0.65,0.75,0.85,0.95])).sum()

In [0]:
print("Simple Model Confidence Error : ",calGap(total_bin, count_bin))

In [0]:
print("Ensemble Model Confidence Error ",calGap(total_bin_ensemble, count_bin_ensemble))



---

